<a href="https://colab.research.google.com/github/ssrbazpur/Data-Science/blob/master/python3_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import sys
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()

In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/Data Science/nyc-taxi-265120-baf9a3e4cf9b.json"
%reload_ext google.cloud.bigquery

In [0]:
#@title Setup Project Variables { run: "auto", display-mode: "form" }
project_id = 'nyc-taxi-265120' #@param {type:"string"}
dataset_name = "Taxitemp" #@param {type:"string"}
model_name = "pats_taxi_model" #@param {type:"string"}
eval_name = model_name + "_eval"
client = bigquery.Client(project=project_id)

In [0]:

project_id = 'nyc-taxi-265120' #@param{type:"string"}
from google.cloud import bigquery
#from google.cloud import bigquery_storage_v1beta1
# Create BigQuery client
client = bigquery.Client(project = project_id)
# Create BigQuery storage client
#bq_storage_client = bigquery_storage_v1beta1.BigQueryStorageClient()

In [0]:
query = """
with wd as (
    SELECT 
        cast(year as STRING) as year,
        EXTRACT (DAYOFYEAR FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP)) AS daynumber, 
        MIN(EXTRACT (DAYOFWEEK FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP))) dayofweek,
        MIN(min) mintemp, MAX(max) maxtemp, MAX(IF(prcp=99.99,0,prcp)) rain
    FROM `bigquery-public-data.noaa_gsod.gsod2019`
    WHERE stn='725030'   --station id 725030=LaGuardia
    GROUP BY 1,2 
  
  -- TAXI DATA
  ),
TD AS(

SELECT  zone_id, time, pickups ,CAST(EXTRACT (YEAR from time) AS STRING) AS year,EXTRACT (DAYOFYEAR from time) AS daynumber,EXTRACT (HOUR from time) AS hour from `nyc-taxi-265120.hello.test_table` ),points AS
(SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`)


SELECT 
points.zone_id,
td.time,
td.daynumber,
td.hour,
cast(wd.dayofweek as STRING) as dayofweek, 
    wd.mintemp, 
    wd.maxtemp,
    wd.rain,
 td.pickups,

    ST_ASTEXT(points.p) AS P,
 
  FROM wd, td INNER JOIN points ON points.zone_id=td.zone_id
  where wd.year = td.year AND
  wd.daynumber = td.daynumber
  group by zone_id,time,daynumber,hour,pickups,P,dayofweek, mintemp, maxtemp, rain

"""
df2 = client.query(query).to_dataframe()


In [0]:
df2.drop('P', axis=1, inplace=True)

In [0]:
df2.drop('time', axis=1, inplace=True)

In [0]:
df2["zone_id"] = df2["zone_id"].astype(int)
df2["dayofweek"] = df2["dayofweek"].astype(int)

In [0]:
train=df2

In [0]:
train=train.drop('pickups', axis=1)

In [0]:
y=df2['pickups']
X=train

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [0]:
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [14]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
rf_predict = rf.predict(X_test)

In [16]:
from sklearn.metrics import mean_squared_error
print('The rmse of prediction is:', mean_squared_error(y_test, rf_predict) ** 0.5)

The rmse of prediction is: 15.049623257539745


In [0]:
import pickle
pickle.dump(rf, open('/content/drive/My Drive/Data Science/modelfinalprotocol.pkl','wb'),protocol=-1)

In [0]:
 model = pickle.load(open('/content/drive/My Drive/Data Science/modelfinalprotocol.pkl','rb'))


In [21]:
print(model.predict([[1,1,1,0,0,0,0]]))

[0.37]
